## BBC - Project
### Colorectal Cancer Biomarker Discovery - Statistical Analysis
_Samuel Mayor, Jérémie Châtillon, Alexandra Korukova_

The __goal__ of this part of the project is to select the reasonable amount of genes which can potentially serve as the __biomarkers__ for the colorectal cancer discovery.  

Our initial dataset is stored in `./data/GSE21510_family.soft` file and contains the information about the study of the colorectal cancer using the microarray technique.

This notebook contains 3 sections: 
1. __Data Exploration__, where the values of the gene expression are extracted from `./data/GSE21510_family.soft` file
2. __Significance Analysis of Microarrays (SAM)__, where the data to run the SAM tool is generated, then applied to reduce the dataset and get rid of insignificant genes.   
3. __Recursive Feature Extraction + Support Vecor Machines__, where the data generated in the previous section is used to extract the significant genes (according to the tool used)

## 1. Data Exploration

In [1]:
DATA_PATH = './data/'

In [2]:
import numpy as np
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records
    
records = load_geo(DATA_PATH + 'GSE21510_family.soft')

The following cell should be executed multiple times to run through the data

In [3]:
nr = next(records)
print(nr)
print('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
print(nr.entity_attributes)

GEO Type: DATABASE
GEO Id: GeoMiame
Database_email: geo@ncbi.nlm.nih.gov

Database_institute: NCBI NLM NIH

Database_name: Gene Expression Omnibus (GEO)

Database_web_link: http://www.ncbi.nlm.nih.gov/geo

Column Header Definitions


---------------------- ENTITY_ATTRIBUTES:------------------------------

{'Database_name': 'Gene Expression Omnibus (GEO)', 'Database_institute': 'NCBI NLM NIH', 'Database_web_link': 'http://www.ncbi.nlm.nih.gov/geo', 'Database_email': 'geo@ncbi.nlm.nih.gov'}


In [4]:
# re-loading the data
records = load_geo(DATA_PATH + 'GSE21510_family.soft')
series_sample_id = []
sample_titles = []
genes = []
nb_cols = 0
nb_rows = 0
data = []
for r in records:
    rea = r.entity_attributes
    if 'Series_geo_accession' in rea:
        if rea['Series_geo_accession'] == 'GSE21510':
            series_sample_id = rea['Series_sample_id']
            nb_cols = len(series_sample_id)
    if 'Sample_title' in rea:
        sample_titles.append(rea['Sample_title'])
        if 'sample_table_begin' in rea:
            nb_rows = rea['Sample_data_row_count'] 
            data.append(r.table_rows)
                    

The cell below generates the dataset used along the project:  
__data__: a 2D numpy array of shape (54676, 149) == (NB_GENES+1, NB_SAMPLES+1).  
First row will contain the sample ids, first column will contain the ID_REFs of the genes.

In [5]:
all_data = np.ndarray((int(nb_rows)+1, int(nb_cols)+1), dtype=object)
# labels
all_data[0, 0] = 'ID_REF'
all_data[0, 1:] = np.array(series_sample_id)

for i, d in enumerate(data):
    values = np.array(d[1:])
    if (i == 0):
        all_data[1:, 0] = values[:, 0]
    all_data[1:, i+1] = values[:, 1]
data = np.array(all_data)

In [6]:
data.shape

(54676, 149)

The cell below __labels the data__: for every sample of the dataset, assign the label 'cancer' if the corresponding patient has cancer and 'normal' otherwise.  
Result: the dictionnary containing sample ids as keys and labels as values

In [7]:
import re

samples = {}
patient_names = []
for i_t, title in enumerate(sample_titles):
    split = title.split(',')
    if(re.search('cancer', title)):
        samples[series_sample_id[i_t]] = 'cancer'
    else:
        samples[series_sample_id[i_t]] = 'normal'
        
    patient_names.append(split[0])
        
samples

{'GSM537330': 'cancer',
 'GSM537331': 'cancer',
 'GSM537332': 'cancer',
 'GSM537333': 'cancer',
 'GSM537334': 'cancer',
 'GSM537335': 'cancer',
 'GSM537336': 'cancer',
 'GSM537337': 'cancer',
 'GSM537338': 'cancer',
 'GSM537339': 'cancer',
 'GSM537340': 'cancer',
 'GSM537341': 'cancer',
 'GSM537342': 'cancer',
 'GSM537343': 'cancer',
 'GSM537344': 'cancer',
 'GSM537345': 'cancer',
 'GSM537346': 'cancer',
 'GSM537347': 'cancer',
 'GSM537348': 'cancer',
 'GSM537349': 'cancer',
 'GSM537350': 'cancer',
 'GSM537351': 'cancer',
 'GSM537352': 'cancer',
 'GSM537353': 'cancer',
 'GSM537354': 'cancer',
 'GSM537355': 'cancer',
 'GSM537356': 'cancer',
 'GSM537357': 'cancer',
 'GSM537358': 'cancer',
 'GSM537359': 'cancer',
 'GSM537360': 'cancer',
 'GSM537361': 'cancer',
 'GSM537362': 'cancer',
 'GSM537363': 'cancer',
 'GSM537364': 'cancer',
 'GSM537365': 'cancer',
 'GSM537366': 'cancer',
 'GSM537367': 'cancer',
 'GSM537368': 'cancer',
 'GSM537369': 'cancer',
 'GSM537370': 'cancer',
 'GSM537371': 'c

## 2. Significance Analysis of Microarrays (SAM)  

__[Significance Analysis of Microarrays](https://statweb.stanford.edu/~tibs/SAM/)__ is a statistical technique for retrieving the significant genes from a large dataset. SAM allows to retrieve the desired number of significant genes by specifying the __Delta__ parameter. The __False Discovery Rate__ or __FDR__ is the metrics used to evaluate the significance of the gene, similar to the p-value explored in the BBC laboratory, and adapted to the analysis of the large amount of genes.  

In our project, we have set the SAM tool to perform the __two class unpaired__ evaluation. "Two class" means that our dataset is composed of Benign and Malign samples and "unpaired" means that the cancerous and healthy samples are taken from the different patients.  

The __goal__ of this part of the statistical analysis is to purge our dataset and get rid of the insignificant genes. 
We have selected around 22'000 genes to continue the biomarker discovery with a smaller dataset.  

The __output__ of SAM tool contains a lot of information, including:
* __negative and positive significant genes__ for a given Delta parameter  
* Delta table, containing different statistical values for a range of Delta values 
* The SAM plot displayed below.  
Every point on the plot represents a gene. The red points correspond to the positive signigicant genes and the green ones correspond to the negative significant genes. The width of the band between the dotted lines is twice the Delta value. We have set this parameter to ~4.7 to retrieve ~22'000 genes. FDR is equal to 0 for the chosen value of Delta.

![](./img/SamPlot.png)  

*NOTES:* 
* To run SAM, we have used the R package 
* The input for SAM can be found in `./data/colorectal_cancer.xlsx` file
* The SAM output can be found in `./data/colorectal_cancer_res_22k.xlsx` file

### Prepare the data structure to run the SAM tests

The following cells generate an Excel spreadsheet in a format required to run SAM tool:  

![](./img/SAM-input.png)  

The first two cells of the first line must be blank; the following cells contain the labels: 2 for cancer and 1 for healthy patients.  
The first column contains the gene name and the second one contains the gene id.  
All other cells contain the values of the gene expression corresponding to a given gene in a given sample.

In [8]:
import pandas as pd

The Excel files containing data are already generated -> commented

In [9]:
def to_sam_data(data):
    '''
    This function creates the numpy array in the format described above
    from the initial dataset
    '''
    sam_data = np.ndarray((int(nb_rows)+1, int(nb_cols)+2), dtype=object)
    sam_data[:, 1:] = data
    # create the first column: gene IDs
    for i in np.arange(1, sam_data.shape[0]):
        sam_data[i, 0] = 'g' + str(i)
    sam_data[0, 1] = None
    for i_c, c in enumerate(sam_data[0, 2:]):
        if(samples[c] == 'cancer'):
            sam_data[0, i_c+2] = 2
        else:
            sam_data[0, i_c+2] = 1
    return sam_data

The following two cells generate the Excel file in a format required to run the SAM tool.  
The result can be found in `./data/colorectal_cancer_res_22k.xlsx` file.  

In [10]:
# sam_data = to_sam_data(data)

In [11]:
# sam_frame = pd.DataFrame(sam_data)
# writer = pd.ExcelWriter('colorectal_cancer.xlsx')
# sam_frame.to_excel(writer, sheet_name='cancer', index=False, header=False)
# writer.save()

In [12]:
def get_sig_genes(sig_genes_frame, data):
    '''
    This function extracts the significant genes from the dataset
    sig_genes_frame: pandas dataframe containing values of significant genes (SAM result)
    data: the original dataset
    '''
    # extract the indices of all significant genes
    idx_sig_genes = sig_genes_frame.values[:, 0]
    # create a numpy array that will contain the data of all significant genes
    # nb of rows: nb of significant genes
    # nb of cols: nb of samples in the original dataset
    sig_genes = np.ndarray((idx_sig_genes.shape[0]+1, data.shape[1]), dtype=object)
    # add the header line
    sig_genes[0, :] = data[0, :]
    # fill the rest with the ID_REF of the gene (first col) and the gene expression values
    for i, sig_index in enumerate(idx_sig_genes):
        sig_genes[i+1, :] = data[sig_index-1, :]
    return sig_genes

In [13]:
def merge_sigenes(pos_sig_genes, neg_sig_genes, sample_labels):
    '''
    This function merges arrays containing data of the positive and negative significant genes 
    and creates the array for the Trefle classifier
    '''
    # the shape is the sum of number of lines of arrays containing negative and positive significant genes -1 
    # (because both contain the header with the sample names)
    sig_genes_all = np.ndarray((pos_sig_genes.shape[0]+neg_sig_genes.shape[0]-1, pos_sig_genes.shape[1]), dtype=object)
    sig_genes_all[0:pos_sig_genes.shape[0], :] = pos_sig_genes
    sig_genes_all[pos_sig_genes.shape[0]:, :] = neg_sig_genes[1:, :] # without header line

    # Transpose, so every line becomes a vector of features
    # Reserve one column for the labels
    sig_genes_t = np.ndarray((pos_sig_genes.shape[1], pos_sig_genes.shape[0]+neg_sig_genes.shape[0]), dtype=object)
    sig_genes_t[:, 0:sig_genes_t.shape[1]-1] = sig_genes_all.T

    # Label the samples: Malign = 1, Benign = 0
    for i, sample in enumerate(sig_genes_t):
        if (i == 0):
            sig_genes_t[i, sig_genes_t.shape[1]-1] = 'label'
        else:
            if(sample_labels[sig_genes_t[i, 0]] == 'cancer'):
                sig_genes_t[i, sig_genes_t.shape[1]-1] = 1
            else:
                sig_genes_t[i, sig_genes_t.shape[1]-1] = 0
    return sig_genes_t

### Prepare the data structure from the SAM output

Now we will prepare the dataset containing ~22k significant genes extracted with SAM. This dataset will be later used for the RFE feature extraction method

In [14]:
# Read from previously generated Excel file to the pandas dataframe
pos_sig_genes_22k_df = pd.read_excel(open('./data/pos_sigenes_22k.xlsx', 'rb'))
neg_sig_genes_22k_df = pd.read_excel(open('./data/neg_sigenes_22k.xlsx', 'rb'))

# Extract the values corresponding to the significant genes from the original dataset
pos_sig_genes_22k = get_sig_genes(pos_sig_genes_22k_df, data)
neg_sig_genes_22k = get_sig_genes(neg_sig_genes_22k_df, data)

# Merge positive and negative significant genes into single array
sig_genes_all_22k = merge_sigenes(pos_sig_genes_22k, neg_sig_genes_22k, samples)

# RFE method can be applied with this array (with the use of functions below) 
sig_genes_all_22k = sig_genes_all_22k.T # transpose
sig_genes_all_22k = sig_genes_all_22k[0:sig_genes_all_22k.shape[0]-1, :] # drop the last line containing labels

In [15]:
sig_genes_all_22k.shape

(22212, 149)

## 3. Support Vector Machines + Recursive Feature Elimination

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4175386/

Once the dataset is reduced with the SAM method, we can select the significant genes using __Recursive Feature Extraction__ algorithm. It is a feature selection method which consists in considering recursively a smaller and smaller sets of features. At every iteration of the algorithm, the __estimator__ evaluates the importance of every feature in the remaining set and removes _k_ least significant ones. RFE allows to specify the resulting number of selected features and the step (*k*, the number of features to eliminate on every step of the algorithm). In our case, the gene expression values corresponding to one gene is considered a feature. We have selected 400 genes from the previosly generated dataset with the step of 100. This number of resulting significant features was chosen to keep the execution time of the next stage of our project reasonable (see Trefle part).  

The __estimator__ used for evaluating the importance of the certain gene expression values is the __Support Vector Machine__ model, more precisely the C-Support Vector Classification type of the SVM with the linear kernel. SVM is a type of the supervised Machine Learning methods, based on the splitting the data into groups, maximizing the margin between the groups.  

*NOTE*: the result of this step used for the Trefle part of the project can be found in `./data/rfe_result.csv` file.

### Prepare the data structrure for RFE

In [16]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

Transform the data to feed it to the RFE selector

X is an input for RFE: every line of X represents a sample (patient). Every column represents the feature.
y contains the labels for every sample: 1 if the patient has cancer, 0 otherwise 

In [17]:
def rfe_data(data, sample_labels):
    '''
    data: numpy array every column of which represents a sample and the row represents a feature
        first column contains the feature id (ID_REF of the gene in our case)
        first row contains the sample id 
    sample_labels: dictionnary having sample ids as keys and 'cancer'/'no cancer' as values
    returns (X, y) tuple where X contains the values of the feature set
        every line of X contains values corresponding to one sample (sample in our case)
        columns of X represent the features (genes in our case)
        y contains the labels (1-Malign, 0-Benign)
    '''
    X = data[1:, 1:].T
    y = np.zeros(len(sample_labels))
    
    for i, y_i in enumerate(y):
        if(sample_labels[data[0, i+1]] == 'cancer'):
            y[i] = 1
        else:
            y[i] = 0
    return (X, y)

In [18]:
X, y = rfe_data(sig_genes_all_22k, samples)

### Run the RFE algorithm

In [19]:
# Create the RFE object and rank each gene
svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=400, step=100)
# The following function will filter the input so that only the significant {n_features_to_select} or less genes remain
rfe_selected = rfe.fit_transform(X, y)

List the indices of the selected genes

In [20]:
selected_indices = rfe.get_support(indices=True)
selected_indices

array([    0,     1,     2,     3,     4,     6,     7,    10,    11,
          12,    16,    17,    20,    22,    25,    28,    29,    30,
          31,    41,    43,    44,    45,    49,    50,    55,    64,
          80,    85,    94,   103,   109,   113,   121,   127,   129,
         132,   140,   150,   162,   163,   181,   191,   209,   212,
         214,   227,   233,   235,   243,   244,   272,   274,   319,
         324,   333,   354,   366,   373,   378,   401,   406,   424,
         445,   450,   467,   476,   481,   488,   490,   523,   542,
         567,   615,   627,   641,   642,   643,   649,   658,   669,
         675,   683,   694,   798,   831,   844,   865,   875,   965,
         971,   983,  1022,  1052,  1081,  1104,  1114,  1133,  1138,
        1141,  1182,  1198,  1209,  1322,  1333,  1342,  1410,  1420,
        1463,  1532,  1564,  1596,  1766,  1799,  1842,  1864,  1871,
        1938,  2016,  2050,  2070,  2163,  2191,  2334,  2369,  2416,
        2461,  2528,

### Prepare the data for the Trefle Classifier

In [21]:
idx_selected = selected_indices+1 # maka a shift because the first col contains the sample names
nrows = X.shape[0]+1 # +1 line for the sample ids
ncols = selected_indices.shape[0]+2 # +2 columns for the gene id and the label
rfe_selected_data = np.ndarray((nrows, ncols), dtype=object)
rfe_selected_data[:, 0] = data[0,:ncols] # sample ids

# Label the samples: Malign = 1, Benign = 0
b_cnt = 0
for i, sample in enumerate(rfe_selected_data):
    if (i == 0):
        rfe_selected_data[i, ncols-1] = 'label'
    else:
        if(samples[rfe_selected_data[i, 0]] == 'cancer'):
            rfe_selected_data[i, ncols-1] = 1
        else:
            b_cnt += 1
            rfe_selected_data[i, ncols-1] = 0
print(b_cnt)

# gene ids
rfe_selected_data[0, 1:ncols-1] = data[idx_selected,0]
# Now fill the array with the values corresponding to the selected genes
rfe_selected_data[1:, 1:ncols-1] = rfe_selected
rfe_selected_df = pd.DataFrame(rfe_selected_data)
rfe_selected_df

25


,0,1,2,3,4,5,6,7,8,9,...,392,393,394,395,396,397,398,399,400,401
0,ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1316_at,1320_at,1438_at,1487_at,...,210783_x_at,211377_x_at,211461_at,211581_x_at,211585_at,211602_s_at,211616_s_at,212043_at,212612_at,label
1,GSM537330,11.18204744,8.654007946,11.5958136,10.78885867,9.970058164,10.30946487,10.79228985,12.2049361,9.631743521,...,5.591213226,6.227322056,7.392751757,6.224789113,5.689202384,8.825054163,4.5774541,8.677586148,9.52411921,1
2,GSM537331,12.05889929,8.570927803,8.078051196,10.84165751,8.926294779,8.541125393,10.28592502,11.92396692,8.738657178,...,9.389268231,6.628290673,9.206933288,6.919133655,5.44737371,7.733068859,5.68360146,9.253717764,10.64291367,1
3,GSM537332,12.20760134,8.88127098,12.00316315,11.15447693,9.328659717,9.705434261,10.1458872,12.70236567,9.278909508,...,4.672494423,5.974763894,8.415521277,6.363705641,5.779687126,8.974140417,4.366214864,9.455260142,9.140912995,1
4,GSM537333,10.10776161,9.669961284,11.52090695,9.931037961,8.992218311,9.969552761,9.541108789,11.43061019,9.14319428,...,9.102809899,6.592707195,8.624147947,5.925591381,6.856468708,11.26411019,5.386560672,9.393032415,9.602141554,1
5,GSM537334,11.83347823,8.54325762,9.102151499,10.32935868,9.553853044,9.499728856,10.19866325,10.22286291,9.413488425,...,7.871904622,7.019065264,8.827480851,5.830366773,5.798082039,7.738825142,6.570926242,8.909785192,10.46381864,1
6,GSM537335,12.05951236,9.114890763,9.414644981,11.48184062,9.314749297,10.09920405,11.13265214,9.454139017,9.404364981,...,8.267092178,7.85044615,8.996063599,6.575039074,5.34535887,8.087454399,6.809127323,9.028036362,9.936006819,1
7,GSM537336,7.525914339,8.076110964,12.36997219,10.06114487,7.938799139,9.617497437,8.953276983,11.71047557,7.626420622,...,4.811251213,7.139248116,8.729923653,7.404631813,7.54427196,10.18273804,10.03320836,8.806878733,7.929835109,1
8,GSM537337,10.95448265,8.840053413,9.808921861,10.42909422,8.758957342,9.05050804,10.16450644,11.4751955,8.652624617,...,5.516149878,6.856876812,8.231140808,7.793435743,5.582625566,6.426627973,5.495323596,9.87955883,10.11734796,1
9,GSM537338,12.30607122,8.421929106,8.836538943,10.6527966,8.953626302,9.639116871,10.42768544,11.31486494,8.829740187,...,7.816269057,6.968128079,8.239821565,6.885398002,5.907605086,6.889094351,6.094550218,8.703073655,10.10424099,1


In [22]:
# save the result into the csv file
rfe_selected_df.to_csv('rfe_result1.csv', index=False, header=False)

## Conclusion

This part of the project has let us discover two new technologies: SAM and RFE.  
The model built with the data generated in this notebook has the good performance.
The only difficulties we have encountered was the installation of the SAM tool and the data manipulation to generate the input to it.  
If the time would let us, it would be interesting to:
* Extract the data using SAM and RFE with different proportions and explore the results. For example, extract values of expression of ~10'000 genes and then apply the RFE method.
* Build a prediction model, for example a KNN 
* Find the minimum number of significant genes that would result in a good prediction model